# Исследование алгоритма Нелдера-Мида

## Методология

Изучение изменений параметров алгоритма проходит следующим образом:
- Для поиска минимума взята функция Розенброка: $f(x,y) = (1-x)^2 + 100(y-x^2)^2$;
- Формируется множество начальных симплексов;
- Формируется множество параметров алгоритма;
- Для каждой пары "начальный симплекс"-"параметры алгоритма" запускается алгоритм; будет работать, пока "дисперсия" не будет меньше 0.005
- Фиксируем число итераций, сравниваем для начального симплекса по параметрам (и наоборот???)

## Исследование

Подключаем библиотеки:

In [10]:
#r ".\Lib\bin\Debug\net7.0\Lib.dll"

using Lib.Math;
using Lib.Common;
using Lib.Helpers;
using System.Collections.Generic;
using System.Linq;

Формируем набор начальных симплексов --- сгенерируем 10:

In [11]:
static IEnumerable<Point> GenerateRandomPoints(uint count, uint dimension) => Lib.Utilities.Generate(count, () => Lib.Utilities.RandomPoint(dimension));

var simplexes = new List<Simplex> { new Simplex(new List<Point> {
new Point(24.102655164637937, -3.053878934011067),
new Point(-20.38163042362179, 31.530944410677748),
new Point(8.805621455264003, -47.41554003469417),
}),
new Simplex(new List<Point> {
new Point(-43.103066413680445, 49.22256902577887),
new Point(-3.0978742717807535, -30.692589228701404),
new Point(-17.726882768434734, -33.70641216128855),
}),
new Simplex(new List<Point> {
new Point(1.411164037482692, -31.039421913102117),
new Point(-9.385691919989569, -14.600486605794885),
new Point(-26.06245018897566, -9.090523349400002),
}),
new Simplex(new List<Point> {
new Point(24.021904498794683, -18.730667747104523),
new Point(41.260695603234225, -32.779458272699856),
new Point(20.950420261984675, 10.718525977794947),
}),
new Simplex(new List<Point> {
new Point(16.884400841616227, -22.90670931082437),
new Point(11.084396067812385, 9.847208669936478),
new Point(-41.730517746545445, 31.93041356102222),
}),
new Simplex(new List<Point> {
new Point(-23.312101607354506, 13.74578182951415),
new Point(38.37213459928557, 17.95722620601296),
new Point(-33.767026777030466, 45.73163988617614),
}),
new Simplex(new List<Point> {
new Point(0.7580829617789817, 20.777695828115327),
new Point(4.402898662929125, 15.885798639239383),
new Point(-15.132437415683818, -5.473246137767433),
}),
new Simplex(new List<Point> {
new Point(-48.27223745974405, 26.62645298482424),
new Point(1.5106782230896911, -44.829946514431825),
new Point(3.147873313043192, 4.7888794692775605),
}),
new Simplex(new List<Point> {
new Point(-28.62158470043463, -22.106302201427162),
new Point(27.43904875472974, 22.756903541714536),
new Point(-21.92276214773836, -14.930748234045808),
}),
new Simplex(new List<Point> {
new Point(4.109320742529455, -19.775644430808104),
new Point(-44.751687093989226, 14.794556595943106),
new Point(28.857810779938717, -45.09675925865718)})};
simplexes

index value 0 index value 0 [ 24.102655164637937, -3.053878934011067 ] 1 [ -20.38163042362179, 31.530944410677748 ] 2 [ 8.805621455264003, -47.41554003469417 ] 1 index value 0 [ -43.103066413680445, 49.22256902577887 ] 1 [ -3.0978742717807535, -30.692589228701404 ] 2 [ -17.726882768434734, -33.70641216128855 ] 2 index value 0 [ 1.411164037482692, -31.039421913102117 ] 1 [ -9.385691919989569, -14.600486605794885 ] 2 [ -26.06245018897566, -9.090523349400002 ] 3 index value 0 [ 24.021904498794683, -18.730667747104523 ] 1 [ 41.260695603234225, -32.779458272699856 ] 2 [ 20.950420261984675, 10.718525977794947 ] 4 index value 0 [ 16.884400841616227, -22.90670931082437 ] 1 [ 11.084396067812385, 9.847208669936478 ] 2 [ -41.730517746545445, 31.93041356102222 ] 5 index value 0 [ -23.312101607354506, 13.74578182951415 ] 1 [ 38.37213459928557, 17.95722620601296 ] 2 [ -33.767026777030466, 45.73163988617614 ] 6 index value 0 [ 0.7580829617789817, 20.777695828115327 ] 1 [ 4.402898662929125, 15.885798639239383 ] 2 [ -15.132437415683818, -5.473246137767433 ] 7 index value 0 [ -48.27223745974405, 26.62645298482424 ] 1 [ 1.5106782230896911, -44.829946514431825 ] 2 [ 3.147873313043192, 4.7888794692775605 ] 8 index value 0 [ -28.62158470043463, -22.106302201427162 ] 1 [ 27.43904875472974, 22.756903541714536 ] 2 [ -21.92276214773836, -14.930748234045808 ] 9 index value 0 [ 4.109320742529455, -19.775644430808104 ] 1 [ -44.751687093989226, 14.794556595943106 ] 2 [ 28.857810779938717, -45.09675925865718 ]

Рассмотрим следующие параметры алгоритма:

| Название           | Отражение $\alpha$ | Растяжение $\gamma$ | Сжатие $\beta$ |
| ------------------ | ------------------- | -------------------- | --------------- |
| Классика           | 1                   | 2                    | 0.5             |
| Сильное растяжение | 1                   | 2.5                  | 0.5             |
| Слабое растяжение  | 1                   | 1.5                  | 0.5             |
| Сильное отражение  | 1.25                | 2                    | 0.5             |
| Слабое отражение   | 0.75                | 2                    | 0.5             |
| Сильное сжатие     | 1                   | 2                    | 0.75            |
| Слабое сжатие      | 1                   | 2                    | 0.25            |

In [12]:
var coefficients = new List<Lib.Coefficients> {
	new Lib.Coefficients { Reflection = 1,    Expansion = 2,   Shrink = 0.5  },
	new Lib.Coefficients { Reflection = 1,    Expansion = 2.5, Shrink = 0.5  },
	new Lib.Coefficients { Reflection = 1,    Expansion = 1.5, Shrink = 0.5  },
	new Lib.Coefficients { Reflection = 1.25, Expansion = 2,   Shrink = 0.5  },
	new Lib.Coefficients { Reflection = 0.75, Expansion = 2,   Shrink = 0.5  },
	new Lib.Coefficients { Reflection = 1,    Expansion = 2,   Shrink = 0.75 },
	new Lib.Coefficients { Reflection = 1,    Expansion = 2,   Shrink = 0.25 },
};

Определим функции, которые упростят проведение исследования:

In [13]:
(Point, uint) RunSample(
	RealMultivariableFunction function,
	Simplex initialSimplex,
	Lib.Coefficients coefficients,
	double epsilon
)
{
	var method = new Lib.NelderMeadMethod(coefficients, EvaluationStrategyCollection.LastVarianceIsLessThan(epsilon));
	var solution = method.FindMinimum(function, new RealCoordinateSpace(2),initialSimplex, new Lib.Helpers.EmptyLogger(), out var statistics);
	return (solution, statistics.IterationCount);
}

(Point, uint)[,] RunResearch(
	List<Simplex> initialSimplexes,
	List<Lib.Coefficients> coefficientsSet,
	RealMultivariableFunction function,
	double epsilon)
{
	var data = new (Point, uint)[initialSimplexes.Count, coefficientsSet.Count];

	for (var i = 0; i < initialSimplexes.Count; i++)
	{
		var simplex = initialSimplexes[i];
		
		for (var j = 0; j < coefficientsSet.Count; j++)
		{
			var coefficients = coefficientsSet[j];

			data[i,j] = RunSample(function, simplex, coefficients, epsilon);
		}
	}

	return data;
}

Соберём данные:

In [14]:
public sealed class Rosenbrock : RealMultivariableFunction
{
	private readonly double _a;
	private readonly double _b;

	public Rosenbrock(double a, double b)
	{
		_a = a;
		_b = b;
	}

	public static Rosenbrock Classic() => new(1, 100);

	public override uint Dimension => 2;

	protected override double BaseCalculate(Point point)
	{
		var x = point[0];
		var y = point[1];

		return System.Math.Pow(_a - x, 2) + _b * System.Math.Pow(y - System.Math.Pow(x, 2), 2);
	}
}

In [15]:
var data = RunResearch(simplexes, coefficients, Rosenbrock.Classic(), 0.005);

## Анализ полученных данных

Отметим, что решением будет точка $(0, 0)$ --- нам так же важна близость к ней.

In [16]:
data

index value 0 ((-2,3858605603316936, 5,65771992246168), 33) Item1 [ -2.3858605603316936, 5.65771992246168 ] Item2 33 1 ((-2,3858605603316936, 5,65771992246168), 33) Item1 [ -2.3858605603316936, 5.65771992246168 ] Item2 33 2 ((-0,4880244826737413, 0,19846293632365913), 116) Item1 [ -0.4880244826737413, 0.19846293632365913 ] Item2 116 3 ((-0,22906006450030147, 0,05626958853937575), 78) Item1 [ -0.22906006450030147, 0.05626958853937575 ] Item2 78 4 ((-4,4065998602333725, 19,40470163262161), 51) Item1 [ -4.4065998602333725, 19.40470163262161 ] Item2 51 5 ((1,2643348607377982, 1,5998547165131662), 105) Item1 [ 1.2643348607377982, 1.5998547165131662 ] Item2 105 6 ((-1,3685041520545664, 1,8352239151043945), 13) Item1 [ -1.3685041520545664, 1.8352239151043945 ] Item2 13 7 ((-0,2781520404910398, 0,05483953065367464), 104) Item1 [ -0.2781520404910398, 0.05483953065367464 ] Item2 104 8 ((-0,49662457628237244, 0,21991618624135312), 55) Item1 [ -0.49662457628237244, 0.21991618624135312 ] Item2 55 9 ((-0,6333409730242874, 0,3925141098638005), 93) Item1 [ -0.6333409730242874, 0.3925141098638005 ] Item2 93 10 ((-0,2998662794626431, 0,0890708498521439), 43) Item1 [ -0.2998662794626431, 0.0890708498521439 ] Item2 43 11 ((-1,1852323274626562, 1,366109289772866), 73) Item1 [ -1.1852323274626562, 1.366109289772866 ] Item2 73 12 ((0,22296655085791375, -0,005285057101775469), 51) Item1 [ 0.22296655085791375, -0.005285057101775469 ] Item2 51 13 ((-1,7560556469284803, 3,0582492360897167), 54) Item1 [ -1.7560556469284803, 3.0582492360897167 ] Item2 54 14 ((1,0769543018031027, 1,166475617841986), 29) Item1 [ 1.0769543018031027, 1.166475617841986 ] Item2 29 15 ((1,0470798484066655, 1,0985469374316144), 28) Item1 [ 1.0470798484066655, 1.0985469374316144 ] Item2 28 16 ((1,5787675512507158, 2,483676300667212), 28) Item1 [ 1.5787675512507158, 2.483676300667212 ] Item2 28 17 ((-0,4100245741826577, 0,132927220329231), 60) Item1 [ -0.4100245741826577, 0.132927220329231 ] Item2 60 18 ((1,2479756760279916, 1,5495218285418737), 38) Item1 [ 1.2479756760279916, 1.5495218285418737 ] Item2 38 19 ((-0,15325224523841507, -0,017369357456140877), 68) Item1 [ -0.15325224523841507, -0.017369357456140877 ] Item2 68 (50 more)

Посмотрим, какой набор коэффициентов нашёл лучшее решение:

In [27]:
var theBestSolution = new Point(0, 0);

double Distance(Point first, Point second) =>
	Math.Sqrt( Math.Pow(first[0] - second[0], 2) + Math.Pow(first[1] - second[1], 2) );

public IEnumerable<T> GetColumn<T>(T[,] matrix, int columnNumber)
{
	return Enumerable.Range(0, matrix.GetLength(0))
			.Select(x => matrix[x, columnNumber]);
}

public IEnumerable<T> GetRow<T>(T[,] matrix, int rowNumber)
{
	return Enumerable.Range(0, matrix.GetLength(1))
			.Select(x => matrix[rowNumber, x]);
}

var bestSolutions = new List<(Point point, double distance, int idx)>();

for (var i = 0; i < data.GetLength(0); i++)
{
	var row = GetRow(data, i).ToArray();
	var points = row.Select(x => x.Item1).ToArray();
	var bestPoint = points.MinBy(point => Distance(point, theBestSolution));
	bestSolutions.Add((bestPoint, Distance(bestPoint, theBestSolution), Array.IndexOf(points, bestPoint)));
}

bestSolutions

index value 0 ((-0,22906006450030147, 0,05626958853937575), 0,2358702604044709, 3) Item1 [ -0.22906006450030147, 0.05626958853937575 ] Item2 0.2358702604044709 Item3 3 1 ((0,22296655085791375, -0,005285057101775469), 0,22302917887586743, 5) Item1 [ 0.22296655085791375, -0.005285057101775469 ] Item2 0.22302917887586743 Item3 5 2 ((-0,15325224523841507, -0,017369357456140877), 0,1542334115846969, 5) Item1 [ -0.15325224523841507, -0.017369357456140877 ] Item2 0.1542334115846969 Item3 5 3 ((0,032448799979948914, 0,021008616498080307), 0,03865600325049325, 5) Item1 [ 0.032448799979948914, 0.021008616498080307 ] Item2 0.03865600325049325 Item3 5 4 ((0,1702034913433347, -0,019638033209300007), 0,1713326612581214, 5) Item1 [ 0.1702034913433347, -0.019638033209300007 ] Item2 0.1713326612581214 Item3 5 5 ((1,1536704920120868, 1,341742294354259), 1,7695276173596277, 5) Item1 [ 1.1536704920120868, 1.341742294354259 ] Item2 1.7695276173596277 Item3 5 6 ((-0,6694592850869182, 0,45979279833598075), 0,8121484789068557, 5) Item1 [ -0.6694592850869182, 0.45979279833598075 ] Item2 0.8121484789068557 Item3 5 7 ((-0,26927838022963335, 0,044826820660121225), 0,27298404698734624, 0) Item1 [ -0.26927838022963335, 0.044826820660121225 ] Item2 0.27298404698734624 Item3 0 8 ((-0,24513019749656045, 0,04352169812723733), 0,24896375626299724, 1) Item1 [ -0.24513019749656045, 0.04352169812723733 ] Item2 0.24896375626299724 Item3 1 9 ((0,18350703078511044, 0,021354378635117167), 0,1847453377881548, 0) Item1 [ 0.18350703078511044, 0.021354378635117167 ] Item2 0.1847453377881548 Item3 0